In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
import numpy as np
from handle_dataset.transform import create_df_with_datetimes

In [2]:
from ml_models.lightgbm import lightgbm

In [3]:
train_df = pd.read_csv('Dataset/Yearly-train.csv')
test_df = pd.read_csv('Dataset/Yearly-test.csv')

In [4]:

counter = 0

train = create_df_with_datetimes(train_df, counter)
test = create_df_with_datetimes(test_df, counter)

test['target'][3] = 1

# Instantiate and fit the model
light_gbm_forecasts = lightgbm(train, test, 2)

/home/george/.local/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: at least one array or dtype is required

In [14]:
light_gbm_forecasts[1]

,timestamp,target
1,01-01-80 18:00,7290.2
2,01-01-81 00:00,7392.6
3,01-01-82 06:00,7527.6
4,01-01-83 12:00,7594.8
5,01-01-84 18:00,7720.7
6,01-01-85 00:00,7823.2


In [15]:
from evaluation_protocol.grubbs import grubbs_score
from evaluation_protocol.mape import mape
from evaluation_protocol.smape import smape
from evaluation_protocol.shape_similarity import dtw

In [16]:
alpha = 0.05
predicted = light_gbm_forecasts[1]['target']
real = test['target']
grubbs_test_score = grubbs_score(predicted, real, alpha)
smape_score = smape(real.tolist(), predicted.tolist())
shape_similarity_score = dtw(predicted, real)
mape_score = mape(real.tolist(), predicted.tolist())

model_name = 'light_gbm'

print(f"Grubbs score for {model_name} is : {grubbs_test_score}")
print(f"SMAPE score for {model_name} is : {smape_score}")
print(f"MAPE score for {model_name} is : {mape_score}")
print(f"Shape similarity score for {model_name} is : {shape_similarity_score}")
print("===================================================================================")

Grubbs score for light_gbm is : 0
SMAPE score for light_gbm is : 0.0
MAPE score for light_gbm is : 0.0
Shape similarity score for light_gbm is : 0.0


In [17]:
predicted

1    7290.2
2    7392.6
3    7527.6
4    7594.8
5    7720.7
6    7823.2
Name: target, dtype: object

In [18]:
real

1    7290.2
2    7392.6
3    7527.6
4    7594.8
5    7720.7
6    7823.2
Name: target, dtype: object